In [1]:
# Setup (OPTIONAL - USED IF NEEDED)
## bind 'set enable-bracketed-paste off'

In [2]:
## TEST:  Make sure simple prompt used (e.g., no escapes that might contaminate output)
## PS1="$ "
## TODO: PS1="> ""
## NOTE: The Jupyter bash kernel requires that PS1 not be modified as they customize it. 
echo $PS1

[PEXP\[\]ECT_PROMPT>


In [3]:
# Delete all aliases and function
# TODO: Instead start from pristine environment
unalias -a
alias | wc -l
for f in $(typeset -f | egrep '^\w+'); do unset -f $f; done
typeset -f | egrep '^\w+' | wc -l

0
0


In [4]:
## NOTE: Source it directly from the ./tests directory.
# Setting a temp directory for tests
TMP=/tmp/test-py-commands
BIN_DIR=$PWD/..
## You will need to run jupyter from that directory.
## source $TEMP_BIN/_dir-aliases.bash
alias | wc -l

0


In [5]:
## NOTE: For reproducability, the directory name needs to be fixed
## In place of $$, use a psuedo random number (e,g., 7766)
## *** All output from one run to the next needs to be the same ***

## temp_dir=$TMP/test-$$
temp_dir=$TMP/test-7766

mkdir -p "$temp_dir"
# TODO: /bin/rm -rvf "$temp_dir"
cd "$temp_dir"
pwd

#ALIAS FOR PRINTING SEPERATION LINES (FOR JUPYTER)
alias linebr="printf '%*s\n' "${COLUMNS:-$(tput cols)}" '' | tr ' ' -"

/tmp/test-py-commands/test-7766


In [6]:
alias testuser="sed -r "s/"$USER"+/user/g""
alias testnum="sed -r "s/[0-9]/X/g""
# alias testmonth="sed -r "s/['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']/M/g""

In [7]:
source $BIN_DIR/tomohara-aliases.bash

In [8]:
# add-python-path $temp_dir
# alias ps-python-full='ps-all python'

# alias ps-python='ps-python-full | $EGREP -iv "(screenlet|ipython|egrep|update-manager|software-properties|networkd-dispatcher)"'
# alias show-python-path='show-path-dir PYTHONPATH'

In [9]:
ps-python-full | wc -l | testnum
# Sample Output: 23

XX


In [10]:
ps-python | wc -l | testnum
# Sample Output: 22

XX


In [11]:
## show-python-path DISPLAYS ALL THE AVAILABLE PATHS OF PYTHON
show-python-path | testuser

PYTHONPATH:
/home/user/python



In [12]:
# function python-lint-work() { python-lint-full "$@" 2>&1 | $EGREP -v '\((bad-continuation|bad-option-value|fixme|invalid-name|locally-disabled|too-few-public-methods|too-many-\S+|trailing-whitespace|star-args|unnecessary-pass)\)' | $EGREP -v '^(([A-Z]:[0-9]+)|(Your code has been rated)|(No config file found)|(\-\-\-\-\-))' | $PAGER; }

# function python-lint() { python-lint-work "$@" 2>&1 | $EGREP -v '(Exactly one space required)|\((bad-continuation|bad-whitespace|bad-indentation|bare-except|c-extension-no-member|consider-using-enumerate|consider-using-with|global-statement|global-variable-not-assigned|keyword-arg-before-vararg|len-as-condition|line-too-long|logging-not-lazy|misplaced-comparison-constant|missing-final-newline|redefined-variable-type|redundant-keyword-arg|superfluous-parens|too-many-arguments|too-many-instance-attributes|trailing-newlines|useless-\S+|wrong-import-order|wrong-import-position)\)' | $PAGER; }


In [13]:
# Takes time for execution
python-lint-work os | testnum

************* Module os
/usr/lib/pythonX.XX/os.py:XX:X: WXXXX: Redefining built-in 'open' (redefined-builtin)
/usr/lib/pythonX.XX/os.py:XX:XX: WXXXX: Redefining name 'name' from outer scope (line XX) (redefined-outer-name)
/usr/lib/pythonX.XX/os.py:XX:X: WXXXX: Wildcard import posix (wildcard-import)
/usr/lib/pythonX.XX/os.py:XX:X: EXXXX: Unable to import 'nt' (import-error)
/usr/lib/pythonX.XX/os.py:XX:X: WXXXX: Wildcard import nt (wildcard-import)
/usr/lib/pythonX.XX/os.py:XX:X: EXXXX: Unable to import 'nt' (import-error)
/usr/lib/pythonX.XX/os.py:XX:X: CXXXX: Import "from os.path import curdir, pardir, sep, pathsep, defpath, extsep, altsep, devnull" should be placed at the top of the module (wrong-import-position)
/usr/lib/pythonX.XX/os.py:XXX:XX: WXXXX: Redefining built-in 'str' (redefined-builtin)
/usr/lib/pythonX.XX/os.py:XXX:XX: WXXXX: Redefining name 'name' from outer scope (line XX) (redefined-outer-name)
/usr/lib/pythonX.XX/os.py:XXX:XX: WXXXX: Redefining name 'name' from out

/usr/lib/pythonX.XX/os.py:XX:X: CXXXX: standard import "from os.path import curdir, pardir, sep, pathsep, defpath, extsep, altsep, devnull" should be placed before "from _collections_abc import _check_methods" (wrong-import-order)
/usr/lib/pythonX.XX/os.py:XX:X: CXXXX: Imports from package posix are not grouped (ungrouped-imports)
/usr/lib/pythonX.XX/os.py:XXX:X: CXXXX: Imports from package _collections_abc are not grouped (ungrouped-imports)




In [14]:
# May take some time
python-lint os | testnum

************* Module os
/usr/lib/pythonX.XX/os.py:XX:X: WXXXX: Redefining built-in 'open' (redefined-builtin)
/usr/lib/pythonX.XX/os.py:XX:XX: WXXXX: Redefining name 'name' from outer scope (line XX) (redefined-outer-name)
/usr/lib/pythonX.XX/os.py:XX:X: WXXXX: Wildcard import posix (wildcard-import)
/usr/lib/pythonX.XX/os.py:XX:X: EXXXX: Unable to import 'nt' (import-error)
/usr/lib/pythonX.XX/os.py:XX:X: WXXXX: Wildcard import nt (wildcard-import)
/usr/lib/pythonX.XX/os.py:XX:X: EXXXX: Unable to import 'nt' (import-error)
/usr/lib/pythonX.XX/os.py:XXX:XX: WXXXX: Redefining built-in 'str' (redefined-builtin)
/usr/lib/pythonX.XX/os.py:XXX:XX: WXXXX: Redefining name 'name' from outer scope (line XX) (redefined-outer-name)
/usr/lib/pythonX.XX/os.py:XXX:XX: WXXXX: Redefining name 'name' from outer scope (line XX) (redefined-outer-name)
/usr/lib/pythonX.XX/os.py:XXX:X: WXXXX: Redefining name 'error' from outer scope (line XX) (redefined-outer-name)
/usr/lib/pythonX.XX/os.py:XXX:XX: WXXXX: 

/usr/lib/pythonX.XX/os.py:XX:X: CXXXX: Imports from package posix are not grouped (ungrouped-imports)
/usr/lib/pythonX.XX/os.py:XXX:X: CXXXX: Imports from package _collections_abc are not grouped (ungrouped-imports)




In [15]:
# ERROR: Pattern not found  (press RETURN)
run-python-lint-batched $BIN_DIR/tests/auto_batspp.py | testuser | testnum

************* Module auto_batspp
/home/user/tom-project/shell-scripts/tests/auto_batspp.py:XX:X: WXXXX: Using an f-string that does not have any interpolated variables (f-string-without-interpolation)
/home/user/tom-project/shell-scripts/tests/auto_batspp.py:XXX:X: WXXXX: Using an f-string that does not have any interpolated variables (f-string-without-interpolation)
/home/user/tom-project/shell-scripts/tests/auto_batspp.py:XXX:X: WXXXX: Using an f-string that does not have any interpolated variables (f-string-without-interpolation)
/home/user/tom-project/shell-scripts/tests/auto_batspp.py:XXX:XX: RXXXX: Boolean condition ''-n' or '-t' not in ARG' will always evaluate to ''-n'' (condition-evals-to-constant)
/home/user/tom-project/shell-scripts/tests/auto_batspp.py:XXX:XX: RXXXX: Boolean condition ''n' or 't' not in ARG' will always evaluate to ''n'' (condition-evals-to-constant)
/home/user/tom-project/shell-scripts/tests/auto_batspp.py:XXX:X: WXXXX: Using an f-string that does not have

In [16]:
## REQUIRES setup.py (TBC)

# python-setup-install
# linebr
# python-uninstall-setup

In [17]:
# mezcla MODULE IS USED FOR TEST

python-import-path 'mezcla' | testnum
linebr
python-import-path-full 'mezcla' | testnum
linebr

# WORKS WELL, SHOWS SOME ISSUES ON BATSPP 2.1.X
python-import-path-all 'mezcla' | grep mezcla | head -n 5 | testnum

matches /usr/local/lib/pythonX.XX/dist-packages/mezcla/__init__.py
--------------------------------------------------------------------------------
matches /usr/local/lib/pythonX.XX/dist-packages/mezcla/__init__.py
matches /usr/local/lib/pythonX.XX/dist-packages/mezcla/debug.py
matches /usr/local/lib/pythonX.XX/dist-packages/mezcla/sys_version_info_hack.py
matches /usr/local/lib/pythonX.XX/dist-packages/mezcla/glue_helpers.py
matches /usr/local/lib/pythonX.XX/dist-packages/mezcla/system.py
matches /usr/local/lib/pythonX.XX/dist-packages/mezcla/tpo_common.py
--------------------------------------------------------------------------------
# /usr/local/lib/pythonX.XX/dist-packages/mezcla/__pycache__/__init__.cpython-XXX.pyc matches /usr/local/lib/pythonX.XX/dist-packages/mezcla/__init__.py
# code object from '/usr/local/lib/pythonX.XX/dist-packages/mezcla/__pycache__/__init__.cpython-XXX.pyc'
# /usr/local/lib/pythonX.XX/dist-packages/mezcla/__pycache__/debug.cpython-XXX.pyc matches /usr/l

In [18]:
# RETURNS A LIST OF PYTHON MODULES
rm -rf ./* > /dev/null
pip3 freeze | grep ipython | wc -l

2


In [19]:
# MODULE USED: mezcla | sed filter replaces versin number with XX
python-module-version-full mezcla | testnum
linebr
python-module-version mezcla | testnum
linebr
python-package-members mezcla | testnum
linebr

X.X.X
--------------------------------------------------------------------------------
X.X.X
--------------------------------------------------------------------------------
['PYTHONX_PLUS', 'TL', '__VERSION__', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'debug', 'gh', 'glue_helpers', 'mezcla', 'sys', 'sys_version_info_hack', 'system', 'tpo_common']
--------------------------------------------------------------------------------


In [20]:
# ## rename-with-file-date REQUIRED
# function rename-with-file-date() {
#     local f new_f
#     local move_command="move"
#     if [ "$1" = "--copy" ]; then

#         move_command="command cp --interactive --verbose --preserve"
#         shift
#     fi
#     for f in "$@"; do
    
#         if [ -e "$f" ]; then
#            new_f=$(get-free-filename "$f".$(date --reference="$f" '+%d%b%y') ".")

#            eval "$move_command" "$f" "$new_f";
#         fi
#     done;

# }

# function get-free-filename() {
#     local base="$1"
#     local sep="$2"
#     local L=1
#     local filename="$base"
#     while [ -e "$filename" ]; do
#         let L++
#         filename="$base$sep$L"
#     done;
#     echo "$filename"
# }

# alias move='mv'
# export PAGER=less
# export PAGER_CHOPPED="less -S"
# export PAGER_NOEXIT="less -+F"
# function zless () { zcat "$@" | $PAGER; }

# function zhead () {
#     local file="$1"
#     shift
#     zcat "$file" | head "$@"
# }

# alias less-='$PAGER_NOEXIT'
# alias less-clipped='$PAGER_NOEXIT -S'
# alias less-tail='$PAGER_NOEXIT +G'
# alias less-tail-clipped='$PAGER_NOEXIT +G -S'
# alias ltc=less-tail-clipped

In [21]:
pwd 
printf "THIS IS THE HEAD\n1\n2\n3\n4\n5\n6\n7\nTHIS IS THE TAIL." >> testless.txt

/tmp/test-py-commands/test-7766


In [22]:
ltc testless.txt | testnum

THIS IS THE HEAD
X
X
X
X
X
X
X
THIS IS THE TAIL.

In [23]:
less-tail testless.txt | testnum

THIS IS THE HEAD
X
X
X
X
X
X
X
THIS IS THE TAIL.

In [24]:
less-tail-clipped testless.txt | testnum

THIS IS THE HEAD
X
X
X
X
X
X
X
THIS IS THE TAIL.

In [25]:
less-clipped testless.txt | testnum

THIS IS THE HEAD
X
X
X
X
X
X
X
THIS IS THE TAIL.

In [26]:
less- testless.txt | testnum

THIS IS THE HEAD
X
X
X
X
X
X
X
THIS IS THE TAIL.

In [27]:
ipython --version | testuser | testnum
linebr
# STATEMENT TRACING WORKS WELL / ISSUES WITH TESTUSER
python-trace l $BIN_DIR/tests/auto_batspp.py | testuser | testnum

]X;ipython [/tmp/test-py-commands/test-XXXX]]X;ipython [/tmp/test-py-commands/test-XXXX]X.X.X
--------------------------------------------------------------------------------
Cannot run file '/home/aveey/tom-project/shell-scripts/tests/../tests/auto_batspp.py' because: [Errno 2] No such file or directory: './batspp-only/'
Command exited with non-zero status 1
0.06user 0.01system 0:00.09elapsed 82%CPU (0avgtext+0avgdata 12568maxresident)k
40inputs+0outputs (0major+1817minor)pagefaults 0swaps
 --- modulename: auto_batspp, funcname: <module>
auto_batspp.py(X): """ 
auto_batspp.py(XX): import os
auto_batspp.py(XX): import sys
auto_batspp.py(XX): IPYNB = ".ipynb"
auto_batspp.py(XX): BATSPP = ".batspp"
auto_batspp.py(XX): BATS = ".bats"
auto_batspp.py(XX): TXT = ".txt"
auto_batspp.py(XX): BATSPP_STORE = r"batspp-only"
auto_batspp.py(XX): BATS_STORE = r"bats-only"
auto_batspp.py(XX): TXT_STORE = r"txt-reports"
auto_batspp.py(XX): KCOV_STORE = r"kcov-output"
auto_batspp.py(XX): files = os.

In [28]:
# which-python SHOWS THE PATH OF PYTHON
which-python
#(ADDED FOR PYTHON3)
alias which-py3='which python3' 
which-py3

/usr/bin/python
/usr/bin/python3


In [29]:
rm -rf ./* > /dev/null
printf "print('THIS IS A TEST')" > atest.py
printf "print('THIS IS A TEST')" > xyz.py
printf "print('THIS IS A TEST11')" > abc1.py

# SHOWS ALL THE DIFFERENCES BETWEEN THE PYTHON SCRIPTS
# WORKS FINE - HALTS TESTS

py-diff atest.py xyz.py | testuser | testnum | awk '!($6="")'

Assuming implicit --no-glob, as otherwise  would be in extension
abcX.py vs. atest.py   
Differences: abcX.py atest.py   
-rw-rw-r-- X user user XX  X XX:XX abcX.py
-rw-rw-r-- X user user XX  X XX:XX atest.py
XcX     
< print('THIS IS A TESTXX') 
\ No newline at end  file
---     
> print('THIS IS A TEST') 
\ No newline at end  file
------------------------------------------------------------------------     
     
atest.py vs. atest.py   
------------------------------------------------------------------------     
     
xyz.py vs. atest.py   
------------------------------------------------------------------------     
     


In [30]:
# # TEST NOT APPLICABLE RN 
# function kivy-win32-env {
#    export PYTHONPATH='c:/cartera-de-tomas/python;c:/Program-Misc/python/kivy-1-9-0/kivy27'
#    kivy_dir="/c/Program-Misc/python/kivy-1-9-0"
#    python_dir="$kivy_dir/Python27"
#    prepend-path "$kivy_dir:$kivy_dir/Python27:$kivy_dir/tools:$kivy_dir/Python27/Scripts:$kivy_dir/gstreamer/bin:$kivy_dir/MinGW/bin:$kivy_dir/SDL2/bin"
# }

In [31]:
# # NEED HELP - MAYBE APPLICABLE?
# alias elide-data='python -m transpose_data --elide'
# alias kill-python="kill_em.sh --filter 'ipython|emacs' python"
# alias kill-python-all="kill_em.sh python"

In [32]:
# # ERRORS = OSTYPE is 'undefined variable' 
# # NOTE: elide-data shows RuntimeError if any testfile is named abc.py

# $ echo $OSTYPE
#   linebr
#   elide-data $BIN_DIR/template.py
#   linebr
#   kill-python
#   linebr
#   kill-python-all

# | linux-gnu
# | --------------------------------------------------------------------------------
# | /home/xaea12/miniconda3/bin/python: No module named transpose_data
# | --------------------------------------------------------------------------------
# | pattern=/:[0-9][0-9] [^ ]*python/
# | filter=/ipython|emacs/
# | OSTYPE: Undefined variable.
# | --------------------------------------------------------------------------------
# | pattern=/:[0-9][0-9] [^ ]*python/
# | filter=/($)(^)/
# | OSTYPE: Undefined variable.


In [33]:
alias TODAY='date'
alias extract-matches='extract_matches.perl'

function run-jupyter-notebook () {
    local port="$1"; if [ "$port" = "" ]; then port=8888; fi
    local ip="$2"; if [ "$ip" = "" ]; then ip="127.0.0.1"; fi
    local log="$TEMP/jupyter-$(TODAY).log"
    jupyter notebook --NotebookApp.token='' --no-browser --port $port --ip $ip >> "$log" 2>&1 &
    echo "$log"
    local delay=5
    echo "sleeping $delay seconds for log to stabalize (effing jupyter)"
    sleep $delay
    tail "$log"
    echo -n "URL: "
    extract-matches 'http:\S+' "$log" | sort -u
}
alias jupyter-notebook-redir=run-jupyter-notebook
alias jupyter-notebook-redir-open='run-jupyter-notebook 8888 0.0.0.0'

In [34]:
# # ERROR GENERATED (see below)
# $ jupyter-notebook-redir-open

# | [1] 5335
# | /jupyter-Sun Aug 28 08:13:40 AM +0545 2022.log
# | sleeping 5 seconds for log to stabalize (effing jupyter)
# | bash: /jupyter-Sun Aug 28 08:13:40 AM +0545 2022.log: Permission denied
# | [1]+  Exit 1                  jupyter notebook --NotebookApp.token='' --no-browser --port $port --ip $ip >> "$log" 2>&1
# | tail: cannot open '/jupyter-Sun Aug 28 08:13:40 AM +0545 2022.log' for reading: No such file or directory
# | URL: Can't open /jupyter-Sun Aug 28 08:13:40 AM +0545 2022.log: No such file or directory at /home/xaea12/bin/extract_matches.perl line 132.
jupyter-notebook-redir-open | testuser | testnum | head -n 5

/home/user/temp/jupyter-Mon Dec  X XX:XX:XX PM +XXXX XXXX.log
sleeping X seconds for log to stabalize (effing jupyter)
[I XXXX-XX-XX XX:XX:XX.XXX LabApp] JupyterLab application directory is /home/user/.local/share/jupyter/lab
[I XX:XX:XX.XXX NotebookApp] The port XXXX is already in use, trying another port.
[I XX:XX:XX.XXX NotebookApp] The port XXXX is already in use, trying another port.


In [35]:
# # ERROR - MODULE NOT FOUND - NEED HELP

# $ rm -rf ./*
#   printf "print('THIS IS A TEST FOR TEXT EXTRACT ALIAS.')" > textex.py
#   xtract-text ./textex.py

# | /home/xaea12/miniconda3/bin/python: No module named extract_document_text

In [36]:
# # ERROR (LOG FILE NOT FOUND) - NEED HELP
# $ test-script ./textex.py
# | abort: no repository found in '/tmp/test-py-commands/test-3443' (.hg not found)
# | bash: tests/_test_textex.03sep22.log: No such file or directory
# | tests/_test_textex.03sep22.log: No such file or directory

In [37]:
# TESTING A PYTHON SCRIPT
printf "print('THIS IS A TEST10')\nprint('THIS IS A TEST11')\nprint('THIS IS A TEST12')\nprint('THIS IS A TEST13')" > random_line_test.py

In [38]:
# # ERROR : MISSING MODULE (No module named randomize_lines)
# # NEED HELP
# randomize-datafile atest.py
randomize-datafile random_line_test.py

print('THIS IS A TEST10')
print('THIS IS A TEST10')
print('THIS IS A TEST11')
print('THIS IS A TEST12')
/usr/bin/python: No module named randomize_lines


In [39]:
# # Need Help (TBF - NEED HELP OPERATING)
# filter-random _r10pct-fubar.data

In [40]:
conditional-source $BIN_DIR/tests/_dir-aliases.bash

touch file1
ln-symbolic file1 link1
linebr
ls

'link1' -> 'file1'
--------------------------------------------------------------------------------
abc1.py  atest.py  file1  link1  random_line_test.py  xyz.py


In [41]:
# TESTS FOR curl-dump AND url-path
curl-dump https://www.example.com/
linebr
ls -l | testuser | testnum | awk '!($6="")'
linebr
url-path $BIN_DIR/tests/auto_batspp.py | testuser

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1256  100  1256    0     0    853      0  0:00:01  0:00:01 --:--:--   853
--------------------------------------------------------------------------------
total XX    
-rw-rw-r-- X user user XX  X XX:XX abcX.py
-rw-rw-r-- X user user XX  X XX:XX atest.py
-rw-rw-r-- X user user X  X XX:XX fileX
lrwxrwxrwx X user user X  X XX:XX linkX -> fileX
-rw-rw-r-- X user user XXX  X XX:XX random_line_test.py
-rw-rw-r-- X user user XXXX  X XX:XX www.example.com
-rw-rw-r-- X user user XX  X XX:XX xyz.py
--------------------------------------------------------------------------------
file:////home/user/tom-project/shell-scripts/tests/auto_batspp.py


In [42]:
# # WORKS WELL - CANNOT USE testnum & testuser (halts test)
# firefox

In [43]:
pwd |  sed -r "s/"$USER"+/user/g"
rm -rf * > /dev/null

/tmp/test-py-commands/test-7766
